<a href="https://colab.research.google.com/github/Baraa710/Premier_League_Predictor/blob/main/Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
ranges = [(n, n+1) for n in range(2006, 2023)]

seasons = ["{}-{}".format(year1,year2) for (year1, year2) in ranges]


In [ ]:
seasons

['2006-2007',
 '2007-2008',
 '2008-2009',
 '2009-2010',
 '2010-2011',
 '2011-2012',
 '2012-2013',
 '2013-2014',
 '2014-2015',
 '2015-2016',
 '2016-2017',
 '2017-2018',
 '2018-2019',
 '2019-2020',
 '2020-2021',
 '2021-2022',
 '2022-2023']

In [ ]:
def get_season_squad_data(season, league = 'pl'):
  try:
    excel_file = pd.ExcelFile('/content/gdrive/MyDrive/APS360_Project/Data_2/{}.xlsx'.format(season))
    if(league !='pl'):
      squad_sheets = [
      'EFL Squad Data stats',
      'EFL Squad Data keepers',
      'EFL Squad Data shooting']
    else:
      squad_sheets = [
      'Squad Data stats',
      'Squad Data keepers',
      'Squad Data shooting']

    dataframes = {} # a dictionary of dataframes, one for each sheet
    squad_data = pd.DataFrame()

    for sheet_name in squad_sheets:
        dataframes[sheet_name] = pd.read_excel(excel_file, sheet_name, header = 1)
        dataframes[sheet_name] = dataframes[sheet_name].drop(index=0)
        if(sheet_name == squad_sheets[0]):
          dataframes[sheet_name] = dataframes[sheet_name][['Squad', '# Pl', 'Gls', 'Ast', 'PK', 'PKatt', 'CrdY', 'CrdR']]
        elif(sheet_name== squad_sheets[1]):
          dataframes[sheet_name] = dataframes[sheet_name][['Squad', 'GA', 'SoTA', 'Saves', 'CS']]
        else:
          dataframes[sheet_name] = dataframes[sheet_name][['Squad','SoT']]

        assert(dataframes[squad_sheets[0]].shape[0]==dataframes[sheet_name].shape[0])
        if sheet_name == squad_sheets[0]:
          squad_data = dataframes[squad_sheets[0]]
        else:
          squad_data = pd.merge(squad_data, dataframes[sheet_name], how = 'inner', on=['Squad'], suffixes=[None, '_{}'.format(sheet_name)])

    squad_data["PremierLeague"] = [int(league=='pl')] * len(squad_data)
    return squad_data

  except ValueError as ve:
      print(f"ValueError occurred for season {season}: {ve}")
      return None


In [ ]:
get_season_squad_data('2022-2023').head(3)

,Squad,# Pl,Gls,Ast,PK,PKatt,CrdY,CrdR,GA,SoTA,Saves,CS,SoT,PremierLeague
0,Arsenal,26.0,84.0,64.0,3.0,4.0,51.0,0.0,43.0,136.0,94.0,14.0,194.0,1
1,Aston Villa,26.0,49.0,35.0,3.0,4.0,80.0,1.0,46.0,151.0,108.0,12.0,145.0,1
2,Bournemouth,31.0,37.0,24.0,0.0,0.0,68.0,0.0,71.0,204.0,136.0,8.0,126.0,1


In [ ]:
get_season_squad_data('2022-2023','efl').head(3)


,Squad,# Pl,Gls,Ast,PK,PKatt,CrdY,CrdR,GA,SoTA,Saves,CS,SoT,PremierLeague
0,Birmingham City,31.0,47.0,35.0,4.0,6.0,93.0,0.0,58.0,173.0,113.0,14.0,144.0,0
1,Blackburn,28.0,50.0,37.0,1.0,2.0,96.0,3.0,54.0,176.0,123.0,15.0,162.0,0
2,Blackpool,35.0,48.0,34.0,3.0,4.0,85.0,9.0,72.0,215.0,147.0,11.0,139.0,0


In [ ]:
squad_seasons_dataframes= {}
for season in seasons:

  squad_seasons_dataframes[season] = pd.concat([get_season_squad_data(season),get_season_squad_data(season,'efl')],ignore_index=True)


In [ ]:
squad_seasons_dataframes['2006-2007']

,Squad,# Pl,Gls,Ast,PK,PKatt,CrdY,CrdR,GA,SoTA,Saves,CS,SoT,PremierLeague
0,Arsenal,25.0,60.0,48.0,10.0,11.0,51.0,3.0,35.0,177.0,142.0,12.0,249.0,1
1,Aston Villa,30.0,45.0,31.0,6.0,8.0,47.0,1.0,41.0,207.0,166.0,13.0,228.0,1
2,Blackburn,29.0,49.0,33.0,5.0,7.0,81.0,8.0,54.0,277.0,223.0,8.0,211.0,1
3,Bolton,23.0,45.0,35.0,6.0,7.0,84.0,4.0,52.0,216.0,164.0,12.0,195.0,1
4,Charlton Ath,29.0,32.0,22.0,4.0,4.0,60.0,4.0,60.0,247.0,187.0,11.0,172.0,1
5,Chelsea,26.0,58.0,49.0,3.0,4.0,65.0,4.0,24.0,175.0,151.0,22.0,235.0,1
6,Everton,26.0,50.0,35.0,8.0,8.0,70.0,2.0,36.0,221.0,185.0,14.0,200.0,1
7,Fulham,25.0,36.0,27.0,3.0,3.0,60.0,3.0,60.0,237.0,177.0,7.0,177.0,1
8,Liverpool,29.0,56.0,44.0,6.0,7.0,43.0,0.0,27.0,173.0,146.0,20.0,249.0,1
9,Manchester City,29.0,28.0,21.0,2.0,4.0,58.0,4.0,44.0,199.0,155.0,14.0,179.0,1


In [ ]:
players_seasons_dataframes = {}
keepers_seasons_dataframes = {}
keeper_col_names = ['Player','Age', 'MP', 'Min', 'GA', 'SoTA', 'Saves', 'CS', 'Starts']
for season in seasons:
  file_path = '/content/gdrive/MyDrive/APS360_Project/Player_data/player_data_cleaned/{}.xlsx'.format(season)
  players_seasons_dataframes[season] = pd.read_excel(file_path, sheet_name='Players').drop(['Unnamed: 0'], axis = 1)
  keepers_seasons_dataframes[season] = pd.read_excel(file_path, sheet_name='keepers', names = keeper_col_names)
  # Group by 'Player' and aggregate statistics
  players_agg_functions = {'Age':'max', 'MP':'sum', 'Min':'sum', 'Gls':'sum', 'Ast':'sum', 'PK':'sum', 'PKatt':'sum', 'CrdY':'sum',
       'CrdR':'sum', 'Starts':'sum'}
  keepers_agg_functions = {'Age': 'max', 'MP': 'sum', 'Min': 'sum', 'GA': 'sum', 'SoTA': 'sum', 'Saves': 'sum', 'CS': 'sum', 'Starts': 'sum'}
  players_seasons_dataframes[season] = players_seasons_dataframes[season].groupby('Player').agg(players_agg_functions).reset_index()
  keepers_seasons_dataframes[season] = keepers_seasons_dataframes[season].groupby('Player').agg(keepers_agg_functions).reset_index()



In [ ]:
players_seasons_dataframes['2006-2007'].columns

Index(['Player', 'Age', 'MP', 'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'CrdY',
       'CrdR', 'Starts'],
      dtype='object')

In [ ]:
# !pip install xlwt
# !mkdir /content/gdrive/MyDrive/APS360_Project/Squad_data/
# excel_file = '/content/gdrive/MyDrive/APS360_Project/Squad_data/squad_data_stats.xls'
# with pd.ExcelWriter(excel_file) as writer:
#     # Iterate over each key-value pair in the dictionary
#     for sheet_name, df in squad_seasons_dataframes.items():
#         # Write each DataFrame to a separate sheet
#         df.to_excel(writer, sheet_name=sheet_name, index=False)

<ipython-input-116-794e116c1c89>:4: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  with pd.ExcelWriter(excel_file) as writer:


In [ ]:
excel_file = pd.ExcelFile('/content/gdrive/MyDrive/APS360_Project/Match_Data/Fixtures_With_Players/2022-2023.xlsx')

fixtures_df= pd.read_excel(excel_file)

In [ ]:
fixtures_df=fixtures_df.drop('Unnamed: 0', axis = 1)
fixtures_df.shape

(380, 47)

In [ ]:
columns_to_drop = [col for col in fixtures_df.columns if col.endswith('-url')]
fixtures_df=fixtures_df.drop(columns=columns_to_drop)
fixtures_df.columns

Index(['Home', 'Away', 'Score', 'Home-Player-1', 'Home-Player-2',
       'Home-Player-3', 'Home-Player-4', 'Home-Player-5', 'Home-Player-6',
       'Home-Player-7', 'Home-Player-8', 'Home-Player-9', 'Home-Player-10',
       'Home-Player-11', 'Away-Player-0', 'Away-Player-1', 'Away-Player-2',
       'Away-Player-3', 'Away-Player-4', 'Away-Player-5', 'Away-Player-6',
       'Away-Player-7', 'Away-Player-8', 'Away-Player-9', 'Away-Player-10'],
      dtype='object')

In [ ]:
fixtures_df.shape

(380, 25)

In [ ]:
def compare_scores(score):
  '''
  Takes score
  If home team won, return 0
  if away team won, return 1
  if draw, return 2
  '''
  x, y = map(int, score.split('–'))
  if x > y:
      return 0
  elif x < y:
      return 1
  else:
      return 2


# fixtures_df['winner'] = fixtures_df['Score'].apply(compare_scores)
# fixtures_df

In [ ]:
fixtures_df['winner'] = fixtures_df['Score'].apply(compare_scores)
fixtures_df

,Home,Away,Score,Home-Player-1,Home-Player-2,Home-Player-3,Home-Player-4,Home-Player-5,Home-Player-6,Home-Player-7,...,Away-Player-2,Away-Player-3,Away-Player-4,Away-Player-5,Away-Player-6,Away-Player-7,Away-Player-8,Away-Player-9,Away-Player-10,winner
0,Crystal Palace,Arsenal,0–2,Vicente Guaita,Tyrick Mitchell,Marc Guéhi,Jordan Ayew,Eberechi Eze,Wilfried Zaha,Jeffrey Schlupp,...,Marc Guéhi,Jordan Ayew,Eberechi Eze,Wilfried Zaha,Jeffrey Schlupp,Joachim Andersen,Nathaniel Clyne,Odsonne Édouard,Cheick Doucouré,1
1,Fulham,Liverpool,2–2,Marek Rodák,Kenny Tete,Tosin Adarabioyo,Harrison Reed,Neeskens Kebano,Aleksandar Mitrović,Tim Ream,...,Tosin Adarabioyo,Harrison Reed,Neeskens Kebano,Aleksandar Mitrović,Tim Ream,Bobby Reid,Andreas Pereira,João Palhinha,Antonee Robinson,2
2,Tottenham,Southampton,4–1,Hugo Lloris,Pierre Højbjerg,Son Heung-min,Harry Kane,Emerson,Eric Dier,Cristian Romero,...,Son Heung-min,Harry Kane,Emerson,Eric Dier,Cristian Romero,Ryan Sessegnon,Dejan Kulusevski,Rodrigo Bentancur,Ben Davies,0
3,Newcastle Utd,Nott'ham Forest,2–0,Nick Pope,Kieran Trippier,Fabian Schär,Joelinton,Callum Wilson,Allan Saint-Maximin,Matt Targett,...,Fabian Schär,Joelinton,Callum Wilson,Allan Saint-Maximin,Matt Targett,Miguel Almirón,Joe Willock,Dan Burn,Bruno Guimarães,0
4,Leeds United,Wolves,2–1,Illan Meslier,Robin Koch,Brenden Aaronson,Marc Roca,Patrick Bamford,Jack Harrison,Tyler Adams,...,Brenden Aaronson,Marc Roca,Patrick Bamford,Jack Harrison,Tyler Adams,Diego Llorente,Rodrigo,Pascal Struijk,Rasmus Kristensen,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,Everton,Bournemouth,1–0,Jordan Pickford,James Tarkowski,Dwight McNeil,Amadou Onana,Demarai Gray,Yerry Mina,Abdoulaye Doucouré,...,Dwight McNeil,Amadou Onana,Demarai Gray,Yerry Mina,Abdoulaye Doucouré,Alex Iwobi,Idrissa Gana Gueye,Conor Coady,James Garner,0
376,Leicester City,West Ham,2–1,Daniel Iversen,Wout Faes,Jonny Evans,Harvey Barnes,Youri Tielemans,James Maddison,Kelechi Iheanacho,...,Jonny Evans,Harvey Barnes,Youri Tielemans,James Maddison,Kelechi Iheanacho,Kiernan Dewsbury-Hall,Timothy Castagne,Luke Thomas,Boubakary Soumaré,0
377,Aston Villa,Brighton,2–1,Emiliano Martínez,Matty Cash,Ezri Konsa,Tyrone Mings,Douglas Luiz,John McGinn,Ollie Watkins,...,Ezri Konsa,Tyrone Mings,Douglas Luiz,John McGinn,Ollie Watkins,Lucas Digne,Leon Bailey,Jacob Ramsey,Boubacar Kamara,0
378,Leeds United,Tottenham,1–4,Joel Robles,Luke Ayling,Adam Forshaw,Robin Koch,Liam Cooper,Jack Harrison,Rodrigo,...,Adam Forshaw,Robin Koch,Liam Cooper,Jack Harrison,Rodrigo,Pascal Struijk,Rasmus Kristensen,Weston McKennie,Maximilian Wöber,1


In [ ]:
fixtures_df.drop(columns=['Score'], inplace=True)

In [ ]:
col_names = (squad_seasons_dataframes['2022-2023'].columns).tolist()
col_names

['Squad',
 '# Pl',
 'Gls',
 'Ast',
 'PK',
 'PKatt',
 'CrdY',
 'CrdR',
 'GA',
 'SoTA',
 'Saves',
 'CS',
 'SoT',
 'PremierLeague']

In [ ]:
# Merge the matches DataFrame with the teams statistics DataFrame for home team
merged_df = pd.merge(fixtures_df, squad_seasons_dataframes['2021-2022'], left_on='Home', right_on='Squad', how='left')
# Merge the resulting DataFrame with the teams statistics DataFrame for away team
merged_df = pd.merge(merged_df, squad_seasons_dataframes['2021-2022'], left_on='Away', right_on='Squad', suffixes=('_Home', '_Away'), how='left')
merged_df


,Home,Away,Home-Player-1,Home-Player-2,Home-Player-3,Home-Player-4,Home-Player-5,Home-Player-6,Home-Player-7,Home-Player-8,...,PK_Away,PKatt_Away,CrdY_Away,CrdR_Away,GA_Away,SoTA_Away,Saves_Away,CS_Away,SoT_Away,PremierLeague_Away
0,Crystal Palace,Arsenal,Vicente Guaita,Tyrick Mitchell,Marc Guéhi,Jordan Ayew,Eberechi Eze,Wilfried Zaha,Jeffrey Schlupp,Joachim Andersen,...,5.0,8.0,67.0,4.0,48.0,147.0,100.0,13.0,186.0,1
1,Fulham,Liverpool,Marek Rodák,Kenny Tete,Tosin Adarabioyo,Harrison Reed,Neeskens Kebano,Aleksandar Mitrović,Tim Ream,Bobby Reid,...,7.0,8.0,50.0,1.0,26.0,107.0,82.0,21.0,237.0,1
2,Tottenham,Southampton,Hugo Lloris,Pierre Højbjerg,Son Heung-min,Harry Kane,Emerson,Eric Dier,Cristian Romero,Ryan Sessegnon,...,4.0,4.0,65.0,2.0,67.0,183.0,118.0,8.0,161.0,1
3,Newcastle Utd,Nott'ham Forest,Nick Pope,Kieran Trippier,Fabian Schär,Joelinton,Callum Wilson,Allan Saint-Maximin,Matt Targett,Miguel Almirón,...,6.0,7.0,88.0,3.0,40.0,169.0,130.0,17.0,189.0,0
4,Leeds United,Wolves,Illan Meslier,Robin Koch,Brenden Aaronson,Marc Roca,Patrick Bamford,Jack Harrison,Tyler Adams,Diego Llorente,...,1.0,1.0,62.0,2.0,43.0,167.0,125.0,11.0,133.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,Everton,Bournemouth,Jordan Pickford,James Tarkowski,Dwight McNeil,Amadou Onana,Demarai Gray,Yerry Mina,Abdoulaye Doucouré,Alex Iwobi,...,4.0,5.0,99.0,5.0,39.0,153.0,117.0,21.0,204.0,0
376,Leicester City,West Ham,Daniel Iversen,Wout Faes,Jonny Evans,Harvey Barnes,Youri Tielemans,James Maddison,Kelechi Iheanacho,Kiernan Dewsbury-Hall,...,4.0,6.0,52.0,3.0,51.0,165.0,113.0,8.0,142.0,1
377,Aston Villa,Brighton,Emiliano Martínez,Matty Cash,Ezri Konsa,Tyrone Mings,Douglas Luiz,John McGinn,Ollie Watkins,Lucas Digne,...,4.0,7.0,72.0,2.0,44.0,146.0,102.0,11.0,141.0,1
378,Leeds United,Tottenham,Joel Robles,Luke Ayling,Adam Forshaw,Robin Koch,Liam Cooper,Jack Harrison,Rodrigo,Pascal Struijk,...,5.0,5.0,69.0,1.0,40.0,137.0,98.0,16.0,185.0,1


In [ ]:
merged_df.columns

Index(['Home', 'Away', 'Home-Player-1', 'Home-Player-2', 'Home-Player-3',
       'Home-Player-4', 'Home-Player-5', 'Home-Player-6', 'Home-Player-7',
       'Home-Player-8', 'Home-Player-9', 'Home-Player-10', 'Home-Player-11',
       'Away-Player-0', 'Away-Player-1', 'Away-Player-2', 'Away-Player-3',
       'Away-Player-4', 'Away-Player-5', 'Away-Player-6', 'Away-Player-7',
       'Away-Player-8', 'Away-Player-9', 'Away-Player-10', 'winner',
       'Squad_Home', '# Pl_Home', 'Gls_Home', 'Ast_Home', 'PK_Home',
       'PKatt_Home', 'CrdY_Home', 'CrdR_Home', 'GA_Home', 'SoTA_Home',
       'Saves_Home', 'CS_Home', 'SoT_Home', 'PremierLeague_Home', 'Squad_Away',
       '# Pl_Away', 'Gls_Away', 'Ast_Away', 'PK_Away', 'PKatt_Away',
       'CrdY_Away', 'CrdR_Away', 'GA_Away', 'SoTA_Away', 'Saves_Away',
       'CS_Away', 'SoT_Away', 'PremierLeague_Away', 'Home_Players',
       'Away_Players'],
      dtype='object')

In [ ]:
def merge_player_data(original_df, season):
  # Create an empty DataFrame to store player statistics
  player_stats_merged = pd.DataFrame()
  merged_df = original_df
  # Iterate over each player column and merge player stats
  for i in range(1, 12):
      home_player_column = f'Home-Player-{i}'
      away_player_column = f'Away-Player-{i-1}'
      player_stats_column_suffix = f'_player{i}'
      seasons_dataframe = players_seasons_dataframes
      if(i==1):
        seasons_dataframe = keepers_seasons_dataframes
      # Merge with the home player
      home_player_stats = pd.merge(merged_df, seasons_dataframe[season], left_on=home_player_column, right_on='Player', how='left', suffixes=(f'_home_player{i}', None))

      # Merge with the away player
      merged_df = pd.merge(home_player_stats, seasons_dataframe[season], left_on=away_player_column, right_on='Player', how='left', suffixes=(f'_away_player{i}', None))

  return merged_df



In [ ]:
merged_df = merge_player_data(merged_df, '2020-2021')

In [ ]:
to_drop = ['Home','Away','Home-Player-1','Home-Player-2','Home-Player-3','Home-Player-4','Home-Player-5','Home-Player-6','Home-Player-7',
 'Home-Player-8','Home-Player-9','Home-Player-10','Home-Player-11','Away-Player-0','Away-Player-1','Away-Player-2','Away-Player-3','Away-Player-4','Away-Player-5','Away-Player-6',
 'Away-Player-7','Away-Player-8','Away-Player-9','Away-Player-10', 'Squad_Home', 'Squad_Away']
merged_df.drop(to_drop, inplace=True, axis = 1)
merged_df.shape

(380, 265)

In [ ]:
cols = [col for col in merged_df.columns]
cols

['winner',
 '# Pl_Home',
 'Gls_Home',
 'Ast_Home',
 'PK_Home',
 'PKatt_Home',
 'CrdY_Home',
 'CrdR_Home',
 'GA_Home',
 'SoTA_Home',
 'Saves_Home',
 'CS_Home',
 'SoT_Home',
 'PremierLeague_Home',
 '# Pl_Away',
 'Gls_Away',
 'Ast_Away',
 'PK_Away',
 'PKatt_Away',
 'CrdY_Away',
 'CrdR_Away',
 'GA_Away',
 'SoTA_Away',
 'Saves_Away',
 'CS_Away',
 'SoT_Away',
 'PremierLeague_Away',
 'Player_away_player1',
 'Age_away_player1',
 'MP_away_player1',
 'Min_away_player1',
 'GA_away_player1',
 'SoTA_away_player1',
 'Saves_away_player1',
 'CS_away_player1',
 'Starts_away_player1',
 'Player_home_player2',
 'Age_home_player2',
 'MP_home_player2',
 'Min_home_player2',
 'GA',
 'SoTA',
 'Saves',
 'CS',
 'Starts_home_player2',
 'Player_away_player2',
 'Age_away_player2',
 'MP_away_player2',
 'Min_away_player2',
 'Gls_away_player2',
 'Ast_away_player2',
 'PK_away_player2',
 'PKatt_away_player2',
 'CrdY_away_player2',
 'CrdR_away_player2',
 'Starts_away_player2',
 'Player_home_player3',
 'Age_home_player3',

In [ ]:
ranges = [(n, n+1) for n in range(2007, 2023)]
seasons = ["{}-{}".format(year1,year2) for (year1, year2) in ranges]
seasons_matchdata = {}

def prev_season(season):
    # Split the season string into two years
    start_year, end_year = season.split('-')

    # Convert the years to integers
    start_year = int(start_year)
    end_year = int(end_year)

    # Calculate the previous season
    previous_start_year = start_year - 1
    previous_end_year = end_year - 1

    # Format the previous season string
    previous_season = f'{previous_start_year}-{previous_end_year}'

    return previous_season

def get_data_for_season(season):
    with pd.ExcelFile(f'/content/gdrive/MyDrive/APS360_Project/Match_Data/Fixtures_With_Players/{season}.xlsx') as excel_file:
        fixtures_df = pd.read_excel(excel_file)
        columns_to_drop = [col for col in fixtures_df.columns if col.endswith('-url')]
        fixtures_df=fixtures_df.drop(columns=columns_to_drop)
        fixtures_df.drop(columns=['Unnamed: 0'], inplace=True)
        fixtures_df['winner'] = fixtures_df['Score'].apply(compare_scores)
        fixtures_df.drop(columns=['Score'], inplace=True)
        merged_df = pd.merge(fixtures_df, squad_seasons_dataframes[prev_season(season)], left_on='Home', right_on='Squad', how='left')
        merged_df = pd.merge(merged_df, squad_seasons_dataframes[prev_season(season)], left_on='Away', right_on='Squad', suffixes=('_Home', '_Away'), how='left')
        merged_df = merge_player_data(merged_df,prev_season(season))
        to_drop = ['Home','Away','Home-Player-1','Home-Player-2','Home-Player-3','Home-Player-4','Home-Player-5','Home-Player-6','Home-Player-7',
            'Home-Player-8','Home-Player-9','Home-Player-10','Home-Player-11','Away-Player-0','Away-Player-1','Away-Player-2','Away-Player-3','Away-Player-4','Away-Player-5','Away-Player-6',
            'Away-Player-7','Away-Player-8','Away-Player-9','Away-Player-10', 'Squad_Home', 'Squad_Away']
        merged_df.drop(columns=to_drop, inplace=True)
        seasons_matchdata[season] = merged_df

for season in seasons:
    get_data_for_season(season)

In [ ]:
seasons_matchdata['2009-2010']

,winner,# Pl_Home,Gls_Home,Ast_Home,PK_Home,PKatt_Home,CrdY_Home,CrdR_Home,GA_Home,SoTA_Home,...,Age,MP,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Starts
0,0,29.0,65.0,45.0,2.0,2.0,50.0,2.0,24.0,144.0,...,30,37,3257,12,10,2,2.0,3,1,37
1,0,29.0,36.0,23.0,5.0,6.0,73.0,5.0,55.0,191.0,...,26,27,1787,2,3,0,0.0,4,1,20
2,1,25.0,54.0,38.0,4.0,5.0,49.0,1.0,37.0,186.0,...,27,27,2319,1,0,0,0.0,2,0,26
3,1,29.0,36.0,25.0,2.0,3.0,57.0,5.0,57.0,212.0,...,20,3,183,0,0,0,0.0,1,0,2
4,1,28.0,40.0,28.0,4.0,7.0,73.0,4.0,60.0,177.0,...,26,14,1110,0,0,0,0.0,2,0,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2,29.0,35.0,17.0,3.0,3.0,70.0,3.0,64.0,212.0,...,0,0,0,0,0,0,0.0,0,0,0
376,2,30.0,41.0,25.0,1.0,3.0,66.0,4.0,45.0,197.0,...,29,37,3195,0,2,0,0.0,2,0,37
377,1,21.0,48.0,35.0,4.0,5.0,53.0,2.0,48.0,186.0,...,26,37,3271,3,3,0,0.0,8,0,37
378,0,23.0,71.0,46.0,8.0,8.0,66.0,6.0,60.0,259.0,...,27,45,3959,2,1,0,0.0,4,1,45


In [ ]:
excel_file = '/content/gdrive/MyDrive/APS360_Project/Match_Data/seasonal_match_players_data.xlsx'
with pd.ExcelWriter(excel_file) as writer:
    # Iterate over each key-value pair in the dictionary
    for sheet_name, df in seasons_matchdata.items():
        # Write each DataFrame to a separate sheet
        df.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
excel_file = '/content/gdrive/MyDrive/APS360_Project/Match_Data/total_match_players_data.xlsx'
combined_df = pd.concat(seasons_matchdata.values(), ignore_index=True)
with pd.ExcelWriter(excel_file) as writer:
  combined_df.to_excel(writer, index=False)

In [ ]:
combined_df.shape

(6079, 265)